## Lab 8: OpenAPI로 변경된 기업 정보를 반영하기

<div class="alert alert-info">
**참고:** _나중에 도움이 될 수도 있는 부분이에요^^_
</div>

#### 변경된 기업 정보 가져오기 - 변경정보를 가져와 최근 변경 사항만 추출하기

In [ ]:
## 기업 정보를 조회하고 dataframe으로 만들기

# 기업 정보에 변경 사항이 있는 기업 추출
import requests, json
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# 변경된 날짜의 범위 만들기
today = datetime.today().strftime('%Y%m%d')
yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y%m%d')

In [ ]:
# 정보가 변경된 상장사들 가져 오기
df_listed = 
df_listed[(df_listed['modify_date']>=yesterday) & (df_listed['modify_date']<=today)]

In [ ]:
df_to_be_updated = df_listed[(df_listed['modify_date']>=yesterday) &  
                             (df_listed['modify_date']<=today)]

<div class="alert alert-info">
**참고:** _나중에 도움이 될 수도 있는 부분이에요^^_
</div>

#### 변경된 기업 정보 가져오기 - 저장한 함수를 이용해서 기업 상세 정보 조회하기

In [ ]:
# 저장된 사용자 함수 사용하기
from my_func import *

# 변경된 기업 정보를 차례로 가져와 dict로 만들고 수정일을 넣고 list로 만들기
import time
updated_details = []
for idx, row in df_to_be_updated.iloc[-3:].iterrows():
    dict_corp = get_corp_details_2(row['corp_code'])
    dict_corp['modify_date'] = row['modify_date']
    updated_details.append(dict_corp)
    time.sleep(3)
updated_details

In [ ]:
# dataframe으로 만들고 불필요한 컬럼 제거
df_updated = pd.DataFrame(updated_details)
df_updated = df_updated[df_updated.columns[2:]]
df_updated

In [ ]:
# pickle로 저장하고 확인하기
df_updated.
df_updated = pd.read_pickle(                   )
df_updated

<div class="alert alert-info">
**참고:** _나중에 도움이 될 수도 있는 부분이에요^^_
</div>

#### 참고: 변경된 기업 정보 DB에 반영하기 - 초기 환경 구성하기

In [ ]:
## DB에 구축하기 위한 환경 설정

# sqlalchemy로 DB 연결 설정
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../working/prices.db', echo=False)

In [ ]:
# 비어 있는 목적 table 만들기
df_updated[df_updated['corp_code'].isnull()].to_sql('corp_details', engine, if_exists='replace')

<div class="alert alert-info">
**참고:** _나중에 도움이 될 수도 있는 부분이에요^^_
</div>

#### 참고: 변경된 기업 정보 DB에 반영하기 - 기존 자료 삭제 후 추가 + SQL 조회

In [ ]:
## 변경 사항을 DB에 반영하기

# 변경 정보를 DB 임시 table 테이블에 저장하기
df_updated.to_sql('corp_details_temp', engine, if_exists='replace')

In [ ]:
# 변경 정보가 있는 레코드 삭제하기
query_txt = '''delete from corp_details
               where corp_code in (select corp_code from corp_details_temp)'''
result = engine.execute(query_txt)

In [ ]:
# 변경 정보를 목적 table에 입력하기
query_txt = '''insert into corp_details
               select * from corp_details_temp'''
result = engine.execute(query_txt)

result.rowcount

In [ ]:
# 조건에 맞춰 조회해 보기
pd.read_sql_query("select * from corp_details where adres like '%서울%'", engine)